In [1]:
import glob
import os
import tempfile
import warnings

import geopandas as gpd
from gplately.tools import plate_isotherm_depth

from lib.assign_regions import assign_regions
from lib.calculate_convergence import run_calculate_convergence
from lib.check_files import (
    check_source_data,
    check_plate_model,
)
from lib.combine_point_data import combine_point_data
from lib.coregister_combined_point_data import run_coregister_combined_point_data
from lib.coregister_crustal_thickness import run_coregister_crustal_thickness
from lib.coregister_magnetic import coregister_magnetic
from lib.coregister_ocean_rasters import run_coregister_ocean_rasters
from lib.create_study_area_polygons import run_create_study_area_polygons
from lib.erodep import calculate_erodep
from lib.generate_unlabelled_points import generate_unlabelled_points
from lib.misc import (
    calculate_slab_flux,
    calculate_water_thickness,
)
from lib.slab_dip import calculate_slab_dip

# Suppress occasional joblib warnings
%env PYTHONWARNINGS=ignore::UserWarning

warnings.simplefilter("ignore", UserWarning)


env: PYTHONWARNINGS=ignore::UserWarning


In [2]:
random_seed = 1234

n_jobs = int(os.environ.get("N_JOBS", 8))
times = range(171)
verbose = False

num_unlabelled = 200  # per timestep

os.environ['CURL_CA_BUNDLE'] = ''


### Input and output files

In [3]:
model_dir = "plate_model"
#check_plate_model(model_dir, verbose=verbose)

data_dir = "source_data"
#check_source_data(data_dir, verbose=verbose)
deposits_filename = os.path.join(data_dir, "deposit_data.csv")
regions_filename = os.path.join(data_dir, "regions.shp")
agegrid_dir = os.path.join(data_dir, "AgeGrids")
sedthick_dir = os.path.join(data_dir, "SedimentThickness")
carbonate_dir = os.path.join(data_dir, "CarbonateThickness")
co2_dir = os.path.join(data_dir, "CrustalCO2")
crustal_thickness_dir = os.path.join(data_dir, "CrustalThickness")
mag_anomaly_filename = os.path.join(
    data_dir,
    "MagneticAnomaly",
    "emag2_upcont_interpolated.nc",
)
subducted_quantities_dir = os.path.join(
    data_dir,
    "SubductedQuantities",
)
erodep_dir = os.path.join(data_dir, "ErosionDeposition")

output_dir = "extracted_data"
os.makedirs(output_dir, exist_ok=True)
subduction_data_filename = os.path.join(output_dir, "subducting_plate_data.csv")
study_area_dir = os.path.join(output_dir, "study_area_polygons")
output_filename = os.path.join(output_dir, "training_data.csv")

feature_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.gpml",
    )
)
rotation_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.rot",
    )
)
static_polygons_filename = os.path.join(
    model_dir,
    "StaticGeometries",
    "StaticPolygons",
    "Clennett_2020_StaticPolygons.gpml",
)


### Subducting plate data

In [4]:
with tempfile.TemporaryDirectory() as subduction_kinematics_dir:
    run_calculate_convergence(
        nprocs=n_jobs,
        min_time=min(times),
        max_time=max(times),
        topology_filenames=feature_filenames,
        rotation_filenames=rotation_filenames,
        output_dir=subduction_kinematics_dir,
        verbose=verbose,
    )

    subduction_data = run_coregister_ocean_rasters(
        nprocs=n_jobs,
        times=times,
        input_data=subduction_kinematics_dir,
        agegrid_dir=agegrid_dir,
        topology_features=feature_filenames,
        rotation_model=rotation_filenames,
        sedthick_dir=sedthick_dir,
        carbonate_dir=carbonate_dir,
        co2_dir=co2_dir,
        subducted_thickness_dir=os.path.join(
            subducted_quantities_dir,
            "plate_thickness",
        ),
        subducted_sediments_dir=os.path.join(
            subducted_quantities_dir,
            "sediment_thickness",
        ),
        subducted_carbonates_dir=os.path.join(
            subducted_quantities_dir,
            "carbonate_thickness",
        ),
        subducted_water_dir=os.path.join(
            subducted_quantities_dir,
            "water_thickness",
        ),
        verbose=verbose,
    )
subduction_data["plate_thickness (m)"] = plate_isotherm_depth(
    subduction_data["seafloor_age (Ma)"],
    maxiter=100,
)
subduction_data = calculate_water_thickness(subduction_data)
subduction_data = calculate_slab_flux(subduction_data)
subduction_data = calculate_slab_dip(subduction_data)
if subduction_data_filename is not None:
    subduction_data.to_csv(subduction_data_filename, index=False)


### Create study area polygons along subduction zones

In [5]:
run_create_study_area_polygons(
    nprocs=n_jobs,
    times=times,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    output_dir=study_area_dir,
    verbose=verbose,
    #return_output=False,
    return_output=True,
)


[                                     name polarity  \
 0            Challenger Plateau NE margin    Right   
 1            Challenger Plateau NE margin    Right   
 2                                            Right   
 3                                            Right   
 4                  Apenninic Front 35-0Ma    Right   
 ..                                    ...      ...   
 110                  Caribbean subduction    Right   
 111                  Caribbean subduction    Right   
 112                       Sandwich Trench     Left   
 113  Mediterranean Subduction, DB 04-2007     Left   
 114  Mediterranean Subduction, DB 04-2007     Left   
 
                                    ftype  plate_id  \
 0                gpml:TopologicalNetwork       833   
 1                gpml:TopologicalNetwork       833   
 2    gpml:TopologicalClosedPlateBoundary       301   
 3    gpml:TopologicalClosedPlateBoundary       301   
 4                gpml:TopologicalNetwork       301   
 ..     

### Generate random unlabelled data points

In [6]:
unlabelled = generate_unlabelled_points(
    times=times,
    input_dir=study_area_dir,
    num=num_unlabelled,
    threads=n_jobs,
    seed=random_seed,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    verbose=verbose,
)


In [7]:
unlabelled

,lon,lat,present_lon,present_lat,age (Ma),source,label,region
40,117.694521,-9.274300,117.694521,-9.274300,0,random,unlabelled,other
41,-72.289776,-46.888041,-72.289776,-46.888041,0,random,unlabelled,SAm
42,-69.434516,-35.847885,-69.434516,-35.847885,0,random,unlabelled,SAm
43,163.087808,-10.635644,163.087808,-10.635644,0,random,unlabelled,other
44,-26.880864,-57.019294,-26.880864,-57.019294,0,random,unlabelled,other
...,...,...,...,...,...,...,...,...
34140,-111.255256,-81.730617,-179.307731,-46.634435,170,random,unlabelled,SAm
34134,43.550673,27.166732,37.250119,36.488104,170,random,unlabelled,other
34196,-142.628548,75.011185,139.559378,69.106330,170,random,unlabelled,other
34156,-162.534245,69.316840,142.512800,53.847479,170,random,unlabelled,other


### Combine labelled deposit/non-deposit data with random unlabelled data

In [8]:
combined_points = combine_point_data(
    deposit_data=deposits_filename,
    unlabelled_data=unlabelled,
    static_polygons=static_polygons_filename,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    study_area_dir=study_area_dir,
    min_time=min(times),
    max_time=max(times),
    n_jobs=n_jobs,
    verbose=verbose,
)
del unlabelled


In [9]:
combined_points

,present_lon,present_lat,age (Ma),label,source,plate_id,lon,lat,overriding_plate_id,region
0,-157.240000,57.050000,1,positive,Diaz-Rodriguez et al. 2021,1073.0,-157.098763,57.213403,101,NaN
1,-69.000000,-37.100000,1,negative,Diaz-Rodriguez et al. 2021,2901.0,-68.941709,-37.200976,29201,NaN
2,-69.000000,-37.100000,1,negative,Diaz-Rodriguez et al. 2021,2901.0,-68.941709,-37.200976,201,NaN
3,-68.294700,-21.318500,1,negative,Diaz-Rodriguez et al. 2021,201.0,-68.227801,-21.419742,201,NaN
4,-158.400000,56.520000,4,positive,Diaz-Rodriguez et al. 2021,1073.0,-157.850240,57.176422,101,NaN
...,...,...,...,...,...,...,...,...,...,...
37315,-179.307731,-46.634435,170,unlabelled,random,NaN,-111.255256,-81.730617,801,SAm
37316,37.250119,36.488104,170,unlabelled,random,NaN,43.550673,27.166732,301,other
37317,139.559378,69.106330,170,unlabelled,random,NaN,-142.628548,75.011185,101,other
37318,142.512800,53.847479,170,unlabelled,random,NaN,-162.534245,69.316840,926,other


### Assign subduction data to point deposit/non-deposit/unlabelled data

In [10]:
coregistered_data = run_coregister_combined_point_data(
    point_data=combined_points,
    subduction_data=subduction_data,
    n_jobs=n_jobs,
    verbose=verbose,
)
del combined_points, subduction_data


In [11]:
coregistered_data

,present_lon,present_lat,age (Ma),label,source,plate_id,lon,lat,overriding_plate_id,region,...,trench_plate_ID,subducting_plate_absolute_velocity (cm/yr),co2_volume (m^3/m^2),convergence_rate_parallel (cm/yr),subducting_plate_ID,arc_trench_distance (km),trench_velocity_parallel (cm/yr),distance_to_trench_edge (degrees),water_thickness (m),distance_to_trench (km)
0,-153.091800,60.031900,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.091800,60.031900,101,NaN,...,111.0,-4.474647,1.519972,1.315352,901.0,99.904462,-1.923335,3.682878,165.457352,266.772338
1,-153.435000,59.362600,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.435000,59.362600,101,NaN,...,111.0,-4.525297,1.532966,1.315352,901.0,99.904462,-1.923335,4.176728,147.647232,228.065321
2,-153.535100,58.859600,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.535100,58.859600,101,NaN,...,111.0,-4.565588,1.508053,1.315352,901.0,99.904462,-1.923335,4.571807,147.647232,193.002186
3,-153.673800,58.770300,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.673800,58.770300,101,NaN,...,111.0,-4.575628,1.508053,1.315352,901.0,99.904462,-1.923335,4.670577,147.647232,191.801989
4,-154.357300,58.452800,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-154.357300,58.452800,101,NaN,...,111.0,-4.625635,1.527804,1.315352,901.0,99.904462,-1.923335,5.164426,143.914673,195.740775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37315,-179.307731,-46.634435,170,unlabelled,random,NaN,-111.255256,-81.730617,801,SAm,...,801.0,-11.594706,2.652662,-7.439680,919.0,217.129724,2.065672,22.052631,543.998535,473.356107
37316,37.250119,36.488104,170,unlabelled,random,NaN,43.550673,27.166732,301,other,...,301.0,-4.720240,2.285264,-1.164967,530.0,99.904462,-0.693739,0.527848,497.617523,53.126499
37317,139.559378,69.106330,170,unlabelled,random,NaN,-142.628548,75.011185,101,other,...,926.0,-1.460309,1.738610,-4.913624,101.0,101.523401,5.881586,0.443581,51.695129,320.204162
37318,142.512800,53.847479,170,unlabelled,random,NaN,-162.534245,69.316840,926,other,...,43400.0,-6.193837,2.449433,-1.900056,926.0,104.185862,-0.661980,3.311940,169.075577,518.276362


### Assign magnetic anomaly and crustal thickness data to point data

In [12]:
coregistered_data = run_coregister_crustal_thickness(
    point_data=coregistered_data,
    input_dir=crustal_thickness_dir,
    n_jobs=n_jobs,
    verbose=verbose,
)
coregistered_data = coregister_magnetic(
    data=coregistered_data,
    filename=mag_anomaly_filename,
    n_jobs=n_jobs,
)


In [13]:
coregistered_data

,present_lon,present_lat,age (Ma),label,source,plate_id,lon,lat,overriding_plate_id,region,...,crustal_thickness_std (m),crustal_thickness_n,crustal_thickness_range (m),magnetic_anomaly_mean (nT),magnetic_anomaly_min (nT),magnetic_anomaly_max (nT),magnetic_anomaly_median (nT),magnetic_anomaly_std (nT),magnetic_anomaly_n,magnetic_anomaly_range (nT)
0,-153.091800,60.031900,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.091800,60.031900,101,NaN,...,1130.804077,224,3295.085938,154.652344,35.811516,282.693878,149.069107,62.332241,57,246.882362
1,-153.435000,59.362600,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.435000,59.362600,101,NaN,...,1129.955444,222,3179.992188,73.226997,67.446449,90.968697,71.927338,5.519875,58,23.522247
2,-153.535100,58.859600,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.535100,58.859600,101,NaN,...,1116.777710,219,2803.796875,137.284744,122.035904,156.593994,135.497070,9.474295,57,34.558090
3,-153.673800,58.770300,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.673800,58.770300,101,NaN,...,1110.676025,219,2763.070312,134.857651,88.042747,158.585251,144.321899,20.285322,56,70.542503
4,-154.357300,58.452800,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-154.357300,58.452800,101,NaN,...,1094.846191,216,2659.718750,34.768494,23.034136,52.113960,34.099560,7.306161,53,29.079824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37315,-179.307731,-46.634435,170,unlabelled,random,NaN,-111.255256,-81.730617,801,SAm,...,894.853088,640,2281.203125,-47.745644,-55.562462,-26.274920,-50.548042,7.527466,44,29.287542
37316,37.250119,36.488104,170,unlabelled,random,NaN,43.550673,27.166732,301,other,...,428.381897,76,2200.000000,-89.174515,-128.204422,-65.533401,-85.494110,15.364982,34,62.671021
37317,139.559378,69.106330,170,unlabelled,random,NaN,-142.628548,75.011185,101,other,...,NaN,0,NaN,75.784271,9.306231,106.699234,80.302704,22.314474,80,97.393003
37318,142.512800,53.847479,170,unlabelled,random,NaN,-162.534245,69.316840,926,other,...,NaN,0,NaN,72.711830,39.225700,106.439354,72.116463,20.911398,46,67.213654


In [14]:
coregistered_data.columns

Index(['present_lon', 'present_lat', 'age (Ma)', 'label', 'source', 'plate_id',
       'lon', 'lat', 'overriding_plate_id', 'region',
       'seafloor_spreading_rate (km/Myr)', 'sediment_thickness (m)',
       'subducting_plate_absolute_velocity_parallel (cm/yr)',
       'convergence_obliquity (degrees)', 'subducted_water_volume (m)',
       'convergence_rate (cm/yr)', 'carbonate_thickness (m)',
       'trench_velocity (cm/yr)', 'subducted_carbonates_volume (m)',
       'seafloor_age (Ma)', 'subducted_plate_volume (m)',
       'trench_velocity_orthogonal (cm/yr)',
       'subducting_plate_absolute_obliquity (degrees)',
       'subducted_sediment_volume (m)', 'trench_velocity_obliquity (degrees)',
       'distance_from_trench_start (degrees)', 'arc_segment_length (degrees)',
       'subducting_plate_absolute_velocity_orthogonal (cm/yr)',
       'plate_thickness (m)', 'trench_normal_angle (degrees)',
       'convergence_rate_orthogonal (cm/yr)', 'slab_flux (m^2/yr)',
       'slab_dip (de

In [ ]:
def load_data(
    data: coregistered_data,
    verbose: bool = False,
    copy: bool = True,
) -> pd.DataFrame:
    if not isinstance(data, pd.DataFrame):
        if verbose:
            print(f"Loading data from file: {data}", file=stderr)
        data = pd.read_csv(data)
    elif copy:
        data = pd.DataFrame(data)
    return data

def extract_erodep(time: float, dir: str = erodep_dir) -> NDArray[np.floating]:
    time = np.around(time)
    if time < 0.0:
        raise ValueError(f"Invalid time: {time}")
    timestep_total = _erodep_timestep(time=time, dir=dir)
    if time == 0.0:
        return timestep_total
    return timestep_total + extract_erodep(time - 1.0, dir=dir)

def _erodep_timestep(time, dir=erodep_dir, dt=1.0):
    if time < 0.0:
        raise ValueError(f"Invalid time: {time}")
    if time == 0.0:
        lats, lons = extract_lat_lon(dir=dir)
        shape = (lats.size, lons.size)
        return np.zeros(shape, dtype=np.float_)

    filenames = _get_erodep_filenames(dir=dir)
    erodep_times = {time_from_filename(_) for _ in filenames}
    erodep_times = np.array(sorted(erodep_times))
    valid_times = erodep_times[time > erodep_times]

    erodep_time = np.max(valid_times)
    filename = filename_from_time(erodep_time, dir=dir)
    with xr.open_dataset(filename) as dset:
        erodep_rate = np.array(dset["erate"])
    erodep_rate *= 1.0e6 * 1.0e-3  # mm/yr to m/Myr
    timestep_total = erodep_rate * dt
    return timestep_total

def calculate_erodep2(
    data: coregistered_data,
    input_dir: erodep_dir,
    weighted: bool = True,
    radius=np.deg2rad(2.5),
    n_jobs: int = 1,
    metric: str = "haversine",
    column_name: str = "erosion (m)",
    clip: bool = True,
    verbose: bool = False,
):
    data = load_data(data, verbose=verbose)

    erodep_col = pd.Series(
        np.full(data.shape[0], np.nan),
        index=data.index,
        name=column_name,
    )
    neigh = None
    if verbose:
        print("", end="", flush=True)
    for age, data_age in data.groupby("age (Ma)"):
        if verbose:
            print(f"\rWorking on {age:0.0f} Ma", end="", flush=True)
        try:
            if age == 0:
                point_erodep = np.full(data_age.shape[0], 0.0)
            else:
                total_erodep = extract_erodep(
                    time=age,
                    dir=input_dir,
                )
                lats, lons = extract_lat_lon(dir=input_dir)
                point_erodep, neigh = interpolate_values(
                    lons=data_age["present_lon"],
                    lats=data_age["present_lat"],
                    grid_values=total_erodep,
                    radius=radius,
                    neigh=neigh,
                    grid_lons=lons,
                    grid_lats=lats,
                    weighted=weighted,
                    n_jobs=n_jobs,
                    metric=metric,
                )
            for i, val in zip(data_age.index, point_erodep):
                erodep_col.at[i] = val
        except Exception as err:
            print("\n", end="", flush=True)
            raise err
    data = data.join(erodep_col)
    if clip:
        data[column_name] = np.clip(-1 * data[column_name], 0.0, np.inf)
    return data


In [ ]:
coregistered_data = calculate_erodep2(
    coregistered_data,
    input_dir=erodep_dir,
    n_jobs=n_jobs,
    column_name="erosion (m)",
    verbose=verbose,
)


### Calculate cumulative erosion

In [15]:
coregistered_data

,present_lon,present_lat,age (Ma),label,source,plate_id,lon,lat,overriding_plate_id,region,...,crustal_thickness_std (m),crustal_thickness_n,crustal_thickness_range (m),magnetic_anomaly_mean (nT),magnetic_anomaly_min (nT),magnetic_anomaly_max (nT),magnetic_anomaly_median (nT),magnetic_anomaly_std (nT),magnetic_anomaly_n,magnetic_anomaly_range (nT)
0,-153.091800,60.031900,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.091800,60.031900,101,NaN,...,1130.804077,224,3295.085938,154.652344,35.811516,282.693878,149.069107,62.332241,57,246.882362
1,-153.435000,59.362600,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.435000,59.362600,101,NaN,...,1129.955444,222,3179.992188,73.226997,67.446449,90.968697,71.927338,5.519875,58,23.522247
2,-153.535100,58.859600,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.535100,58.859600,101,NaN,...,1116.777710,219,2803.796875,137.284744,122.035904,156.593994,135.497070,9.474295,57,34.558090
3,-153.673800,58.770300,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-153.673800,58.770300,101,NaN,...,1110.676025,219,2763.070312,134.857651,88.042747,158.585251,144.321899,20.285322,56,70.542503
4,-154.357300,58.452800,0,negative,Diaz-Rodriguez et al. 2021,1073.0,-154.357300,58.452800,101,NaN,...,1094.846191,216,2659.718750,34.768494,23.034136,52.113960,34.099560,7.306161,53,29.079824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37315,-179.307731,-46.634435,170,unlabelled,random,NaN,-111.255256,-81.730617,801,SAm,...,894.853088,640,2281.203125,-47.745644,-55.562462,-26.274920,-50.548042,7.527466,44,29.287542
37316,37.250119,36.488104,170,unlabelled,random,NaN,43.550673,27.166732,301,other,...,428.381897,76,2200.000000,-89.174515,-128.204422,-65.533401,-85.494110,15.364982,34,62.671021
37317,139.559378,69.106330,170,unlabelled,random,NaN,-142.628548,75.011185,101,other,...,NaN,0,NaN,75.784271,9.306231,106.699234,80.302704,22.314474,80,97.393003
37318,142.512800,53.847479,170,unlabelled,random,NaN,-162.534245,69.316840,926,other,...,NaN,0,NaN,72.711830,39.225700,106.439354,72.116463,20.911398,46,67.213654


In [16]:
coregistered_data.to_csv('coregistered_data_mag.csv',index=False)

In [17]:
coregistered_data = calculate_erodep(
    coregistered_data,
    input_dir=erodep_dir,
    n_jobs=n_jobs,
    column_name="erosion (m)",
    verbose=verbose,
)


banana
(37320, 57)
(37320,)
0 (389, 57)
1 (224, 57)
extract erodep TIME: 1
source_data/ErosionDeposition
['source_data/ErosionDeposition/erodep20Ma.nc', 'source_data/ErosionDeposition/erodep155Ma.nc', 'source_data/ErosionDeposition/erodep85Ma.nc', 'source_data/ErosionDeposition/erodep120Ma.nc', 'source_data/ErosionDeposition/erodep110Ma.nc', 'source_data/ErosionDeposition/erodep130Ma.nc', 'source_data/ErosionDeposition/erodep65Ma.nc', 'source_data/ErosionDeposition/erodep15Ma.nc', 'source_data/ErosionDeposition/erodep60Ma.nc', 'source_data/ErosionDeposition/erodep135Ma.nc', 'source_data/ErosionDeposition/erodep90Ma.nc', 'source_data/ErosionDeposition/erodep50Ma.nc', 'source_data/ErosionDeposition/erodep25Ma.nc', 'source_data/ErosionDeposition/erodep170Ma.nc', 'source_data/ErosionDeposition/erodep100Ma.nc', 'source_data/ErosionDeposition/erodep10Ma.nc', 'source_data/ErosionDeposition/erodep40Ma.nc', 'source_data/ErosionDeposition/erodep45Ma.nc', 'source_data/ErosionDeposition/erodep140M

POINT:ERODEP [-203.1027985  -203.1027985  -203.73902193 -203.73902193 -213.45491342
 -213.45491342 -218.7051428  -218.7051428  -160.02574664 -160.02574664
 -196.94014501 -196.94014501 -172.4747566  -172.4747566  -162.33825859
 -162.33825859 -156.22559735 -156.22559735 -159.52727697 -159.52727697
 -156.71857537 -156.71857537 -156.90413799 -156.90413799 -164.29053541
 -272.39046266 -272.39046266 -233.7617885  -233.7617885  -171.57209732
 -171.57209732 -170.17725948 -170.17725948 -170.43644607 -170.43644607
 -169.19255104 -169.19255104 -169.78215231 -169.78215231 -169.84075126
 -169.84075126 -168.95366311 -168.95366311 -169.43706978 -169.43706978
 -169.03443723 -169.03443723 -172.62893641 -172.62893641 -172.76589396
 -172.76589396 -148.29423744 -286.62418239 -139.65512334 -144.3793937
 -144.3793937  -148.55598575 -148.55598575 -175.28383604 -175.28383604
 -137.54529606 -141.28794775 -127.71829855 -139.95943097 -139.16508639
 -139.16765244 -231.41084397 -135.43416362 -183.50143498 -135.283

extract erodep timestep_total: [[         nan          nan          nan ...          nan          nan
           nan]
 [-40.43935889 -60.82516315 -69.98186895 ...  -9.32522066  -9.36833381
  -40.43935889]
 [-13.77298262 -21.31417344 -25.65811192 ...  -9.36959177  -9.30756388
  -13.77298262]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]]
extract erodep TIME: 3.0
extract erodep timestep_total: [[         nan          nan          nan ...          nan          nan
           nan]
 [-40.43935889 -60.82516315 -69.98186895 ...  -9.32522066  -9.36833381
  -40.43935889]
 [-13.77298262 -21.31417344 -25.65811192 ...  -9.36959177  -9.30756388
  -13.77298262]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan     

POINT:ERODEP [ -543.61925262  -550.73944143  -370.20278939  -330.3207324
  -368.71670587  -367.96477851 -1399.99070247  -212.10925192
  -564.24091681  -506.36902254  -155.8900321   -474.96061252
  -884.77565278  -215.75779102            nan  -160.47593412
   -35.6541975   -532.43333932   -94.75099042  -475.63623625
  -475.63623625            nan  -120.46584367  -100.64078127
  -316.38930674  -575.90562503  -370.59902017            nan
  -450.1080488             nan  -518.43728033  -424.20349381
  -216.79481378  -408.33338     -948.11473695   -67.23758912
   -40.79578702  -187.24153843   -32.65906726  -285.43645264
  -285.43645264   -64.62320085  -348.45167915  -575.91649049
  -777.71972097   -71.84655945  -105.32425526  -180.04109258
  -575.6078455   -238.01862132   -36.13974005  -152.56394573
 -1239.62442877  -254.90706218  -254.90706218  -344.11052395
  -733.53390015  -733.53390015   -56.79167161  -151.43948309
  -151.73710664   -77.67889207  -303.24463376 -1372.56998875
 -1445.40348

[[          nan           nan           nan ...           nan
            nan           nan]
 [-207.93886277 -309.86788411 -354.65064126 ...  -52.36817165
   -52.5837374  -207.93886277]
 [ -58.50013475  -95.72001918 -117.43971161 ...  -37.49274759
   -36.17304107  -58.50013475]
 ...
 [          nan           nan           nan ...           nan
            nan           nan]
 [          nan           nan           nan ...           nan
            nan           nan]
 [          nan           nan           nan ...           nan
            nan           nan]]
(1801,) (3601,)
POINT:ERODEP [-1177.75582922 -1369.28186078 -1343.6708735  -1351.18092743
 -1350.48577017 -1345.33709325  -214.93470195  -170.09556798
  -715.4494374   -214.93470195 -1029.69480182 -2065.59475875
 -2290.07700225  -273.81395489  -307.48875649            nan
            nan  -559.59525497   -93.22494881            nan
            nan   -70.36180766            nan  -279.66660173
  -253.49503161            nan  -100.7917

...

POINT:ERODEP [ -1577.88185606             nan  -6424.17575505  -2542.92404324
  -8613.66912338  -1577.92575977 -13690.90798647  -5429.7951893
  -2668.2171765   -2291.99566218  -5211.62495238   -739.19914391
  -1267.70785553   -934.75372474  -4170.32929891  -2016.91908254
  -2971.14422706  -2626.68966601  -1928.44311434  -6991.3236588
  -1962.9222533   -2507.20566645  -9268.10263583  -4790.70148306
  -1936.43974423  -4884.8449946    -179.38363629   -179.38363629
  -6812.79272664  -2724.36338163  -5236.93740985  -1615.75751233
  -2673.78530951   -435.1322215   -4064.40700729  -2714.47322503
   -458.1442256   -3396.40106143  -2051.22957865  -6409.93169219
  -7460.36317998  -2593.00894694  -8617.86242111  -1096.48968951
             nan  -4030.27798732  -7483.03009815  -1548.51367935
             nan  -2609.26140317  -2469.99486361  -4087.61888185
  -8519.3386504    -482.9286211   -5034.17191029  -6103.23345974
  -1965.71370098  -4402.49109895             nan  -2451.48835003
  -3594.342856

extract erodep timestep_total: [[  0.           0.           0.         ...   0.           0.
    0.        ]
 [-59.45569949 -59.45569949 -59.45569949 ... -59.45569949 -59.45569949
  -59.45569949]
 [-37.87170322 -37.87170322 -37.87170322 ... -37.87170322 -37.87170322
  -37.87170322]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]]
extract erodep TIME: 164.0
extract erodep timestep_total: [[  0.           0.           0.         ...   0.           0.
    0.        ]
 [-59.45569949 -59.45569949 -59.45569949 ... -59.45569949 -59.45569949
  -59.45569949]
 [-37.87170322 -37.87170322 -37.87170322 ... -37.87170322 -37.87170322
  -37.87170322]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...  

POINT:ERODEP [ -2477.72937792   -950.29438334  -3952.09410173  -5858.63603188
   -554.89391673  -2234.7849049  -10589.01222314  -2078.73736451
  -2164.67689414  -5597.86380603             nan  -2077.00375691
  -4360.18126788   -289.51392501  -2510.86420923  -3369.61635062
             nan  -2036.96592476             nan   -664.14390823
  -1235.94786732  -3287.65885175  -5794.0099973  -15313.49437038
 -14605.56963641  -6791.0980004   -2077.76644872  -5526.70405759
  -6812.8334955   -7324.37298053   -447.01239932   -545.47811492
             nan  -1955.45861427  -4374.54261718  -6266.57120805
  -1522.30511542  -5489.63952666  -2006.96419063  -1988.35179855
  -2346.00672871  -8653.26409991  -4582.06391508  -2633.28706565
  -7322.17068234 -18845.3929171   -8047.70003443  -7088.06588335
   -731.43109409  -2998.58452607  -8486.78557576  -3371.57412116
  -4834.8918815   -6354.37044994  -1078.12643909  -7968.3407044
  -2766.13520402             nan  -2884.58358681  -3564.27408837
  -8413.33515

[[           nan            nan            nan ...            nan
             nan            nan]
 [-1631.08638985 -1733.01541118 -1776.29701062 ... -1471.7024923
  -1471.91805804 -1631.08638985]
 [-2161.86226801 -2201.6961071  -2243.45554848 ... -2126.01991037
  -2144.29379875 -2161.86226801]
 ...
 [           nan            nan            nan ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]]
(1801,) (3601,)
POINT:ERODEP [ -4813.38831661   -824.97911139  -7613.36047995  -4709.34597416
   -907.79740129  -2651.9021286   -1379.77653054  -5423.96125622
  -2046.55059989  -1012.3319209   -4661.69057511   -841.10734035
  -1534.99458851  -2143.90070431  -2304.23019624   -496.91016652
  -8134.26582759  -4839.73435519  -6360.3445384    -766.68070878
  -5179.8012351   -5466.53581116  -3700.4024137

extract erodep timestep_total: [[-15.63259569 -15.63259569 -15.63259569 ... -15.63259569 -15.63259569
  -15.63259569]
 [-23.72237069 -23.72237069 -23.72237069 ... -23.72237069 -23.72237069
  -23.72237069]
 [-24.38253026 -26.71247749 -26.71247749 ... -24.38253026 -24.38253026
  -24.38253026]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan          nan ...          nan          nan
           nan]]
extract erodep TIME: 169.0
extract erodep timestep_total: [[-15.63259569 -15.63259569 -15.63259569 ... -15.63259569 -15.63259569
  -15.63259569]
 [-23.72237069 -23.72237069 -23.72237069 ... -23.72237069 -23.72237069
  -23.72237069]
 [-24.38253026 -26.71247749 -26.71247749 ... -24.38253026 -24.38253026
  -24.38253026]
 ...
 [         nan          nan          nan ...          nan          nan
           nan]
 [         nan          nan   

### Assign data to regions

In [18]:
points = gpd.GeoSeries.from_xy(
    coregistered_data["present_lon"],
    coregistered_data["present_lat"],
    index=coregistered_data.index,
)
coregistered_data["region"] = assign_regions(
    points,
    regions=regions_filename,
)
del points


In [19]:
coregistered_data.to_csv(output_filename, index=False)

coregistered_data.groupby(["source", "region", "label"]).size()


source                      region          label     
Diaz-Rodriguez et al. 2021  North America   negative        57
                                            positive       170
                            South America   negative       979
                                            positive       126
random                      East Asia       unlabelled    4549
                            North America   unlabelled    7359
                            Other           unlabelled    4104
                            South America   unlabelled    6234
                            Southeast Asia  unlabelled    6758
                            Tethys          unlabelled    6984
dtype: int64

In [ ]:
#!conda list